In [1]:
import torch

In [2]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)


2024-11-25 16:04:31.078797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732550671.099862 2145706 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732550671.106368 2145706 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 16:04:31.128723: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import evaluate

In [4]:
import numpy as np

In [5]:
from transformers import BertConfig

In [6]:
def create_model():
    num_hidden_layers = 8
    hidden_size = 256
    num_attention_heads = 4

    model = AutoModelForSequenceClassification.from_pretrained(
        f"bert-uncased_L-{num_hidden_layers}_H-{hidden_size}_A-{num_attention_heads}",
        config=BertConfig(
            hidden_size=hidden_size,
            num_hidden_layers=num_hidden_layers,
            num_attention_heads=num_attention_heads,
            intermediate_size=1024,
            num_labels=3,
        )
    )
    model = model.to("cuda")
    return model

In [7]:
tokenizer = AutoTokenizer.from_pretrained("google/mobilebert-uncased")

In [8]:
from datasets import load_dataset

In [9]:
mnli = load_dataset("nyu-mll/multi_nli").select_columns(
    ["premise", "hypothesis", "label"]
)

In [10]:
def tokenize_mnli(batch):
    return tokenizer(
        text=batch["premise"],
        text_pair=batch["hypothesis"],
        truncation=True,
        max_length=512,
    )

In [11]:
tokenized_mnli = mnli.map(tokenize_mnli, batched=True)

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
from torch import nn

In [14]:
import torch.nn.functional as F

In [15]:
from dataclasses import dataclass

In [16]:
@dataclass
class PretrainedDistillationTrainingArguments(TrainingArguments):
    temperature: float = 1.0

In [17]:
class PretrainedDistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model
        self.kl_loss = nn.KLDivLoss(reduction="batchmean")
    
    def compute_loss(self, model, inputs, return_outputs=False):
        student_outputs = model(**inputs)
        student_logits = student_outputs.logits
        with torch.no_grad():
            teacher_outputs = self.teacher_model(**inputs)
            teacher_logits = teacher_outputs.logits
        loss = self.kl_loss(
            F.log_softmax(student_logits / self.args.temperature, dim=-1),
            F.softmax(teacher_logits / self.args.temperature, dim=-1),
        )
        return (loss, student_outputs) if return_outputs else loss

In [18]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_prediction):
    predictions, labels = eval_prediction
    return accuracy.compute(
        predictions=np.argmax(predictions, axis=1),
        references=labels,
    )

In [49]:
del model

In [19]:
model = create_model()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-uncased_L-8_H-256_A-4 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
teacher_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=3,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
teacher_model.load_state_dict(torch.load("mnli-base.pt", weights_only=True))

<All keys matched successfully>

In [22]:
teacher_model = teacher_model.to("cuda")

In [54]:
distillation_training_args = PretrainedDistillationTrainingArguments(
    output_dir=f"tmp/mnli-distillation",
    learning_rate=2e-5,
    warmup_ratio=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    push_to_hub=False,
    temperature=2.0
)

In [55]:
distillation_trainer = PretrainedDistillationTrainer(
    model=model,
    teacher_model=teacher_model,
    args=distillation_training_args,
    train_dataset=tokenized_mnli["train"],
    eval_dataset=tokenized_mnli["validation_matched"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [56]:
distillation_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.164100,0.117291,0.750891
2,0.130900,0.089256,0.771574
3,0.099600,0.082478,0.780438
4,0.091200,0.078804,0.787978
5,0.076500,0.076987,0.790728


TrainOutput(global_step=122720, training_loss=0.1280443973305017, metrics={'train_runtime': 11589.8177, 'train_samples_per_second': 169.417, 'train_steps_per_second': 10.589, 'total_flos': 6106062684418200.0, 'train_loss': 0.1280443973305017, 'epoch': 5.0})

In [23]:
distillation_training_args = PretrainedDistillationTrainingArguments(
    output_dir=f"tmp/mnli-distillation",
    learning_rate=2e-5,
    warmup_ratio=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    push_to_hub=False,
    temperature=2.0
)

In [24]:
distillation_trainer = PretrainedDistillationTrainer(
    model=model,
    teacher_model=teacher_model,
    args=distillation_training_args,
    train_dataset=tokenized_mnli["train"],
    eval_dataset=tokenized_mnli["validation_matched"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [25]:
distillation_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.178500,0.129134,0.737239
2,0.138700,0.091760,0.768619
3,0.102800,0.085499,0.780540
4,0.092200,0.079656,0.787061
5,0.076300,0.074609,0.790830
6,0.067300,0.075373,0.795721
7,0.060900,0.072324,0.792766
8,0.052700,0.072567,0.797147
9,0.049200,0.071928,0.796943
10,0.047900,0.071653,0.799185


TrainOutput(global_step=245440, training_loss=0.09572621322051177, metrics={'train_runtime': 23214.3228, 'train_samples_per_second': 169.164, 'train_steps_per_second': 10.573, 'total_flos': 1.221222873172554e+16, 'train_loss': 0.09572621322051177, 'epoch': 10.0})

In [32]:
teacher_model = AutoModelForSequenceClassification.from_pretrained(
    "google/mobilebert-uncased",
    num_labels=3,
)

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
from lora import wrap_bert_model_with_lora

In [33]:
teacher_model = wrap_bert_model_with_lora(teacher_model, rank=32, alpha=32)

In [34]:
teacher_model.load_state_dict(torch.load("mnli-lora.pt", weights_only=True))

<All keys matched successfully>

In [35]:
teacher_model = teacher_model.to("cuda")

In [36]:
del model
model = create_model()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-uncased_L-8_H-256_A-4 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
distillation_training_args = PretrainedDistillationTrainingArguments(
    output_dir=f"tmp/mnli-distillation",
    learning_rate=2e-5,
    warmup_ratio=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    push_to_hub=False,
    temperature=2.0,
)

In [39]:
distillation_trainer = PretrainedDistillationTrainer(
    model=model,
    teacher_model=teacher_model,
    args=distillation_training_args,
    train_dataset=tokenized_mnli["train"],
    eval_dataset=tokenized_mnli["validation_matched"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [40]:
distillation_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.133700,0.115755,0.749975
2,0.102500,0.089623,0.771268
3,0.080500,0.083055,0.779521
4,0.071200,0.078654,0.784208
5,0.062500,0.076105,0.786551


TrainOutput(global_step=122720, training_loss=0.11017466056144844, metrics={'train_runtime': 9534.1628, 'train_samples_per_second': 205.945, 'train_steps_per_second': 12.872, 'total_flos': 6106062684418200.0, 'train_loss': 0.11017466056144844, 'epoch': 5.0})

In [45]:
del model
model = create_model()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-uncased_L-8_H-256_A-4 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
distillation_training_args = PretrainedDistillationTrainingArguments(
    output_dir=f"tmp/mnli-distillation",
    learning_rate=2e-5,
    warmup_ratio=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    push_to_hub=False,
    temperature=2.0,
)

In [47]:
distillation_trainer = PretrainedDistillationTrainer(
    model=model,
    teacher_model=teacher_model,
    args=distillation_training_args,
    train_dataset=tokenized_mnli["train"],
    eval_dataset=tokenized_mnli["validation_matched"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [48]:
distillation_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.144700,0.126223,0.742333
2,0.105500,0.088217,0.770963
3,0.080800,0.078056,0.783087
4,0.070500,0.074933,0.788589
5,0.058600,0.070393,0.794396
6,0.052200,0.069552,0.801936
7,0.047800,0.066433,0.803770
8,0.044000,0.067268,0.801630
9,0.041100,0.065900,0.802649
10,0.038800,0.065094,0.804177


TrainOutput(global_step=245440, training_loss=0.0793797347349791, metrics={'train_runtime': 18832.395, 'train_samples_per_second': 208.525, 'train_steps_per_second': 13.033, 'total_flos': 1.221222873172554e+16, 'train_loss': 0.0793797347349791, 'epoch': 10.0})

In [20]:
sum([param.numel() for param in model.parameters() if param.requires_grad])

14330371

In [21]:
del model
model = create_model()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-uncased_L-8_H-256_A-4 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
training_args = TrainingArguments(
    output_dir=f"tmp/mnli-compact",
    learning_rate=2e-5,
    warmup_ratio=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    push_to_hub=False,
)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_mnli["train"],
    eval_dataset=tokenized_mnli["validation_matched"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.682700,0.656018,0.726949
2,0.606100,0.589842,0.759959
3,0.548200,0.589222,0.763831
4,0.507600,0.569563,0.777585
5,0.457100,0.572861,0.781049
6,0.413200,0.597069,0.784310
7,0.397800,0.616377,0.781966
8,0.346700,0.650682,0.785023
9,0.340100,0.657627,0.784310
10,0.313000,0.668248,0.784819


TrainOutput(global_step=245440, training_loss=0.4746190580271058, metrics={'train_runtime': 7851.4032, 'train_samples_per_second': 500.168, 'train_steps_per_second': 31.261, 'total_flos': 1.221222873172554e+16, 'train_loss': 0.4746190580271058, 'epoch': 10.0})

In [31]:
del model
model = create_model()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-uncased_L-8_H-256_A-4 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
training_args = TrainingArguments(
    output_dir=f"tmp/sst2-compact",
    learning_rate=2e-5,
    warmup_ratio=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    push_to_hub=False,
)

In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_sst2["train"],
    eval_dataset=tokenized_sst2["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [34]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.335800,0.377583,0.842890
2,0.252400,0.377797,0.870413
3,0.183700,0.439050,0.878440
4,0.175700,0.471805,0.873853
5,0.148900,0.511784,0.872706
6,0.127300,0.548129,0.873853
7,0.114300,0.542696,0.875000
8,0.106300,0.577406,0.872706
9,0.096500,0.601633,0.884174
10,0.084000,0.621975,0.885321


TrainOutput(global_step=42100, training_loss=0.17282748371858211, metrics={'train_runtime': 1113.7587, 'train_samples_per_second': 604.7, 'train_steps_per_second': 37.8, 'total_flos': 907846567642716.0, 'train_loss': 0.17282748371858211, 'epoch': 10.0})

In [39]:
del model
model = create_model()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-uncased_L-8_H-256_A-4 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
training_args = TrainingArguments(
    output_dir=f"tmp/sst2-compact",
    learning_rate=2e-6,
    warmup_ratio=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    push_to_hub=False,
)

In [41]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_sst2["train"],
    eval_dataset=tokenized_sst2["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [42]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.614200,0.597795,0.714450
2,0.444000,0.452075,0.787844
3,0.371500,0.399034,0.818807
4,0.355400,0.382024,0.833716
5,0.320200,0.373130,0.844037
6,0.294700,0.355370,0.855505
7,0.278800,0.353704,0.864679
8,0.263100,0.350249,0.872706
9,0.254300,0.353430,0.877294
10,0.220900,0.357953,0.875000


TrainOutput(global_step=126300, training_loss=0.24467212997158652, metrics={'train_runtime': 3368.1214, 'train_samples_per_second': 599.88, 'train_steps_per_second': 37.499, 'total_flos': 2723487563851080.0, 'train_loss': 0.24467212997158652, 'epoch': 30.0})

In [ ]:
del model
model = create_model()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-uncased_L-8_H-256_A-4 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir=f"tmp/sst2-compact",
    learning_rate=2e-6,
    warmup_ratio=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    push_to_hub=False,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_sst2["train"],
    eval_dataset=tokenized_sst2["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.485200,0.472094,0.779817
2,0.378800,0.407089,0.817661
3,0.337000,0.379842,0.837156
4,0.335200,0.374131,0.841743
5,0.312900,0.371728,0.845183
6,0.293900,0.360264,0.854358
7,0.286400,0.359505,0.862385
8,0.279200,0.355645,0.856651
9,0.277500,0.356874,0.856651
10,0.252900,0.356474,0.860092


TrainOutput(global_step=42100, training_loss=0.34087565510403234, metrics={'train_runtime': 1115.5485, 'train_samples_per_second': 603.73, 'train_steps_per_second': 37.739, 'total_flos': 907846567642716.0, 'train_loss': 0.34087565510403234, 'epoch': 10.0})